In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# Create data

In [2]:
import copy
from liegroups import SE2, SO2

params_true = {'T_1_0': SE2.identity(),
               'T_2_0': SE2(SO2.identity(), -np.array([0.5, 0])),
               'T_3_0': SE2(SO2.identity(), -np.array([1, 0])),
               'T_4_0': SE2(SO2.from_angle(np.pi / 2),
                             -(SO2.from_angle(np.pi / 2) * np.array([1, 0.5]))),
               'T_5_0': SE2(SO2.from_angle(np.pi),
                             -(SO2.from_angle(np.pi) * np.array([0.5, 0.5]))),
               'T_6_0': SE2(SO2.from_angle(-np.pi / 2),
                             -(SO2.from_angle(-np.pi / 2) * np.array([0.5, 0])))}

obs = {'T_1_0': params_true['T_1_0'],
       'T_2_1': params_true['T_2_0'] * params_true['T_1_0'].inv(),
       'T_3_2': params_true['T_3_0'] * params_true['T_2_0'].inv(),
       'T_4_3': params_true['T_4_0'] * params_true['T_3_0'].inv(),
       'T_5_4': params_true['T_5_0'] * params_true['T_4_0'].inv(),
       'T_6_5': params_true['T_6_0'] * params_true['T_5_0'].inv(),
       'T_6_2': params_true['T_6_0'] * params_true['T_2_0'].inv()}

params_init = copy.deepcopy(params_true)
for key in params_init.keys():
    params_init[key] = SE2.exp(5 * np.random.rand(3)) * params_init[key]

# Create residual functions

In [3]:
from pyslam.residuals import PoseResidual, PoseToPoseResidual
from pyslam.utils import invsqrt

prior_stiffness = invsqrt(1e-12 * np.identity(3))
odom_stiffness = invsqrt(1e-3 * np.identity(3))
loop_stiffness = invsqrt(1e-3 * np.identity(3))

residual0 = PoseResidual(obs['T_1_0'], prior_stiffness)
residual0_params = ['T_1_0']

residual1 = PoseToPoseResidual(obs['T_2_1'], odom_stiffness)
residual1_params = ['T_1_0', 'T_2_0']

residual2 = PoseToPoseResidual(obs['T_3_2'], odom_stiffness)
residual2_params = ['T_2_0', 'T_3_0']

residual3 = PoseToPoseResidual(obs['T_4_3'], odom_stiffness)
residual3_params = ['T_3_0', 'T_4_0']

residual4 = PoseToPoseResidual(obs['T_5_4'], odom_stiffness)
residual4_params = ['T_4_0', 'T_5_0']

residual5 = PoseToPoseResidual(obs['T_6_5'], odom_stiffness)
residual5_params = ['T_5_0', 'T_6_0']

residual6 = PoseToPoseResidual(obs['T_6_2'], loop_stiffness)
residual6_params = ['T_2_0', 'T_6_0']

# Set up and solve the problem

In [4]:
from pyslam.problem import Problem, Options

options = Options()
options.allow_nondecreasing_steps = True
options.max_nondecreasing_steps = 3
options.print_iter_summary = True

problem = Problem(options)

problem.add_residual_block(residual0, residual0_params)
problem.add_residual_block(residual1, residual1_params)
problem.add_residual_block(residual2, residual2_params)
problem.add_residual_block(residual3, residual3_params)
problem.add_residual_block(residual4, residual4_params)
problem.add_residual_block(residual5, residual5_params)
# problem.add_residual_block(residual6, residual6_params)

problem.initialize_params(params_init)

params_final = problem.solve()

Iter:   1 | Residual: 9.005973e+12 --> 2.246850e+04
Iter:   2 | Residual: 2.246850e+04 --> 2.155061e+04
Iter:   3 | Residual: 2.155061e+04 --> 1.760671e+04
Iter:   4 | Residual: 1.760671e+04 --> 1.205946e-24
Iterations:   4 | Residual: 9.005973e+12 --> 1.205946e-24


# Check results

In [5]:
print("Initial Error:")
for key in params_true.keys():
    print('{}: {}'.format(key, SE2.log(params_init[key].inv() * params_true[key])))

print()

print("Final Error:")
for key in params_true.keys():
    print('{}: {}'.format(key, SE2.log(params_final[key].inv() * params_true[key])))

Initial Error:
T_1_0: [ 2.04464319  2.63320036  2.62633526]
T_2_0: [ 2.09615263 -0.22112782  2.35389298]
T_3_0: [ 0.9924934  -1.51886488  1.8574807 ]
T_4_0: [-3.07634293  5.30522037 -2.91837642]
T_5_0: [-0.17247785  3.25734534 -0.39098169]
T_6_0: [-2.04578793  0.24274879  2.94261193]

Final Error:
T_1_0: [ 0.  0.  0.]
T_2_0: [ 0. -0.  0.]
T_3_0: [ 0.  0.  0.]
T_4_0: [ 0.  0.  0.]
T_5_0: [-0. -0.  0.]
T_6_0: [ 0. -0.  0.]


# Optional: Compute the covariance of the final parameter estimates

In [6]:
problem.compute_covariance()
print('covariance of T_5_0:\n{}'.format( problem.get_covariance_block('T_5_0','T_5_0') ))

covariance of T_5_0:
[[ 0.01275 -0.00025  0.0045 ]
 [-0.00025  0.00425 -0.0005 ]
 [ 0.0045  -0.0005   0.004  ]]
